In [399]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pickle5 as pickle
import pandas as pd
import requests
import os.path
import re

In [400]:
# Extract page
def extract(URL):
    #headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'}
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    #Download website locally ; Replace the path with where the folder is located on your system
    file_name = '/Users/ummerabab-/Desktop/wg-gesucht/Websites/' + URL.split("https://www.wg-gesucht.de/")[1].split('.html')[0] + '.txt'
    with open(file_name, "wb") as file:
        file.write(page.content)
    
    return soup

# Acquire useful information from HTML tags
def transform(soup):
    panel_body = soup.find('div', class_ = 'panel-body')
    rows = panel_body.find_all('div', class_ = 'row')
    
    details_dict = {}
    
    
    
    #WG-Details
    wg_tag = ''
    gesucht_wird = ''
    person_img_src = ''
    
    for item in rows:
        
        for sub_item in item.find_all('div', class_ = 'col-sm-6'):
            if sub_item.h4.text.strip() == 'Die WG':
                ul_tag = sub_item.find('ul')
                for li in ul_tag.find_all('li'):
                    wg_tag = wg_tag + ';' + re.sub('\s+',' ',(li.text.strip().replace('\n','')))
                
            if sub_item.h4.text.strip() == 'Gesucht wird':
                ul_tag = sub_item.find('ul')
                for li in ul_tag.find_all('li'):
                    gesucht_wird = re.sub('\s+',' ',(li.text.strip().replace('\n','')))
    
         #Person
        if item.find('div', class_ = 'mb10'):
            if item.find('img'):
                person_img_src = item.find('img')['src']
#                 print(person_img_src['src'])


            
            
    details_dict = {'Die WG': wg_tag,
                    'Gesucht wird': gesucht_wird,
                    'Person': person_img_src}
        
    
    #Description
    description_tag = panel_body.find('div', id = 'ad_description_text')
    #print(description_tag)

    zimmer = ''
    lage = ''
    wg_leben = ''
    sonstiges = ''
        
    if description_tag.find('div', id ='freitext_0'):
            zimmer = description_tag.find('p', id ='freitext_0_content').text.strip().replace('\n','').replace('\r','')
    if description_tag.find('div', id ='freitext_1'):
            lage = description_tag.find('p', id ='freitext_1_content').text.strip().replace('\n','').replace('\r','')
    if description_tag.find('div', id ='freitext_2'):
            wg_leben = description_tag.find('p', id ='freitext_2_content').text.strip().replace('\n','').replace('\r','')
    if description_tag.find('div', id ='freitext_3'):
            sonstiges = description_tag.find('p', id ='freitext_3_content').text.strip().replace('\n','').replace('\r','')

            
    details_dict['Zimmer'] = zimmer
    details_dict['Lage'] = lage
    details_dict['WG-Leben'] = wg_leben
    details_dict['Sonstiges'] = sonstiges
        
    #print(desc_dict)
            
    return details_dict

In [438]:
#Specify the URL to extract from
URL = "https://www.wg-gesucht.de/wg-zimmer-in-Passau-Hacklberg.6544525.html"

# Call functions to Extract HTML from URL and Transform it.
page = extract(URL)
details_dict = transform(page)
print(details_dict)

{'Die WG': ';12m² Zimmer in 5er WG;;5er WG ( 1 Mann );Bewohneralter: 24 bis 24 Jahre;Rauchen auf dem Balkon erlaubt;Studenten-WG, Berufstätigen-WG, keine Zweck-WG, gemischte WG;Sprache/n: Deutsch, Englisch', 'Gesucht wird': 'Geschlecht egal zwischen 18 und 30 Jahren', 'Person': 't_ph.html?qrup=grap36&id=8738917', 'Zimmer': 'Ich suche zum 01.09.2021 eine*n Nachmieter*in für mein gemütliches Zimmer (Zimmer 4) in unserer 5er WG. Das Zimmer ist etwa 12 qm groß und das Fenster geht raus zu dem Hinterhof, der zum Haus dazugehört, sodass es nachts angenehm ruhig ist. Das Zimmer ist voll möbliert, ausgestattet mit großem Bett, Schrank, Kommode, Regalen und Schreibtisch, die vom Vermieter gestellt werden. Da das Zimmer zur Südseite rausgeht, ist es den ganzen Tag lichtdurchflutet. Das Zimmer kostet 380€ / Monat, wobei im Preis Strom, Internet etc. inklusive sind und keine weiteren Nebenkosten auf dich zukommen. Die Wohnung geht über zwei Stockwerke, wobei sich das Zimmer im oberen Stockwerk bef

In [439]:
# Convert dictionary to data_frame
data_frame = pd.DataFrame([details_dict])
print (data_frame)

# Save to file with pickle in csv format
csv_path = '/Users/ummerabab-/Desktop/wg-gesucht/Code/dataset.csv'
if(os.path.isfile(csv_path)):
    data_frame.to_csv(csv_path, mode='a', encoding='utf-8', index= False, header=False) #
else:
    data_frame.to_csv(csv_path, encoding='utf-8', index= False)
#data_frame.to_csv(r'/Users/ummerabab-/Desktop/wg-gesucht/Code/dataset.csv', index = False)

                                              Die WG  \
0  ;12m² Zimmer in 5er WG;;5er WG ( 1 Mann );Bewo...   

                                Gesucht wird  \
0  Geschlecht egal zwischen 18 und 30 Jahren   

                             Person  \
0  t_ph.html?qrup=grap36&id=8738917   

                                              Zimmer  \
0  Ich suche zum 01.09.2021 eine*n Nachmieter*in ...   

                                                Lage  \
0  Die Wohnung liegt in der schönen Lederergasse,...   

                                            WG-Leben  \
0  Da es nicht nur mich ins Ausland verschlägt, s...   

                                           Sonstiges  
0  Noch einmal zusammenfassend:- Zimmer, ca. 12 m...  


In [317]:
# driver = webdriver.Chrome()
# driver.get("https://www.wg-gesucht.de/wg-zimmer-in-Passau.104.0.1.0.html")
# print(driver.title)

In [ ]:
# def transform(driver):
#     #/html/body/div[@id='master_wrapper']/div[@id='site_wrapper']/div[@id='main_content']/div[@id='main_column']/div[@id='liste-details-ad-7368387']/div[@class='row']/div[@class='col-sm-8 card_body']
#     cards = driver.find_element_by_xpath("/html/body/div[@id='master_wrapper']/div[@id='site_wrapper']/div[@id='main_content']/div[@id='main_column']/div[@id='liste-details-ad-7368387']")
#     print(cards)
#     cards.click()
#     #return len(divs)

# transform(driver)